In [8]:
import os
import pandas as pd
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import warnings
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import openpyxl

warnings.filterwarnings('ignore')

# Get the current directory path
curr_dir = os.getcwd()

# Get the file path in the current directory
file_path = os.path.join(curr_dir, 'Dataset-LOXIMVIs.xlsx')
file_str = 'Dataset-LOXIMVIs-'

#read in the excel file
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names

#read in the data from a sheet in the excel file
sheet_name = 'Parental-SLC NP Positive'
df = pd.read_excel(excel_file, sheet_name= sheet_name)

#fill in the NaN values with 0
df.fillna(0, inplace=True)

#isolating the necessary rows
df = df.iloc[2:18] #need to modify index here 
df.reset_index(drop=True, inplace=True)
#display(df)

#Function for flatenning the Y/target/response column in the sheet
replicates = 12
target = df.iloc[:, -replicates:].values.flatten()

In [9]:
#Function for feature matrix based on formulation composition values
def form_composition_matrix(df, replicates, target):
    
    #Creating dataframe for processed data
    df_processed = pd.DataFrame()
    
    if 'Positive' in sheet_name:
        df_processed['Target_NP-Positive'] = target
    elif 'MFI' in sheet_name:
        df_processed['Target_NP-MFI'] = target

    #Creating X/feature matrix using formulation composition
    feature_list = df['TIME_LIPID'].str.split('_').explode().unique().tolist()

    feature_list = [value for value in feature_list if str(value) not in ['24HR','nan']]
    
    # Adding empty columns to the existing dataframe
    df_processed = df_processed.reindex(columns=[*feature_list, *df_processed.columns])
    df_processed.fillna(0, inplace=True)

    #Adding values to feature columns
    row_index = 0
    for row in df['Formulation DETAILS']:
        row_items = row.split(':')
        composition = []
        lipid = []

        for k in row_items:
            i, j = [value for value in k.split(' ') if value != '']
            composition.append(i)
            lipid.append(j)


        for i,j in zip(composition, lipid):
            for feature in feature_list:
                if feature.lower() in j.lower():
                    #column_index = df_test.columns.get_loc(feature)
                    #df_test.iloc[row_index:row_index+12,column_index] = i
                    column_index = df_processed.columns.get_loc(feature)
                    df_processed.iloc[row_index:row_index+replicates,column_index] = i
        row_index += 12

    return df_processed

In [10]:
#Function for one-hot encoding data by lipid composition
def one_hot_lipid_comp(df, replicates, target):
    #Creating dataframe for processed data
    df_processed = pd.DataFrame()

    if 'Positive' in sheet_name:
        df_processed['Target_NP-Positive'] = target
    elif 'MFI' in sheet_name:
        df_processed['Target_NP-MFI'] = target

    #Creating X/feature list
    feature_list = df['TIME_LIPID'].str.split('_').explode().unique().tolist()
    feature_list = [value for value in feature_list if str(value) not in ['24HR','nan']]

    # Adding empty columns to the existing dataframe
    df_processed = df_processed.reindex(columns=[*feature_list, *df_processed.columns])
    df_processed.fillna(0, inplace=True)

    #Adding values to feature columns
    row_index = 0
    for row in df['Formulation DETAILS']:
        row_items = row.split(':')
        composition = []
        lipid = []

        for k in row_items:
            i, j = [value for value in k.split(' ') if value != '']
            composition.append(i)
            lipid.append(j)


        for i,j in zip(composition, lipid):
            for feature in feature_list:
                if feature.lower() in j.lower():
                    column_index = df_processed.columns.get_loc(feature)
                    df_processed.iloc[row_index:row_index+replicates,column_index] = 1
        row_index += 12
    
    return df_processed

In [11]:
#Function for one-hot encoding data by head and tail composition
def one_hot_head_tail_comp(df, replicates, target):
    #Creating dataframe for processed data
    df_processed = pd.DataFrame()

    if 'Positive' in sheet_name:
        df_processed['Target_NP-Positive'] = target
    elif 'MFI' in sheet_name:
        df_processed['Target_NP-MFI'] = target

    #Creating X/feature list
    feature_list = df['TIME_HEADGRPS_CHOLESTEROL_TAIL'].str.split('_').explode().unique().tolist()
    feature_list = [value for value in feature_list if str(value) not in ['24HR','nan']]

    # Adding empty columns to the existing dataframe
    df_processed = df_processed.reindex(columns=[*feature_list, *df_processed.columns])
    df_processed.fillna(0, inplace=True)

    #Adding values to feature columns
    row_index = 0
    for row in df['Formulation DETAILS']:
        row_items = row.split(':')
        composition = []
        lipid = []

        for k in row_items:
            i, j = [value for value in k.split(' ') if value != '']
            composition.append(i)
            lipid.append(j)


        for i,j in zip(composition, lipid):
            for feature in feature_list:
                if feature.lower() in j.lower():
                    column_index = df_processed.columns.get_loc(feature)
                    df_processed.iloc[row_index:row_index+replicates,column_index] = 1
        row_index += 12
    
    return df_processed

In [12]:
def add_more_features(df, df_processed, replicates=12):
    additional_features = dict(zip([0,1,2,3,4,5], df.columns[3:9].to_list()))

    # Ask the user for input
    print('Enter a list of integers separated by commas: for denoting the columns to be used. Ex: 0,1,2,3,4,5')

    #Code updated to include formulation ID number
    
    print('(0)ID (1) Z-Avg, (2) Zeta Potential, (3) PDI, (4) Concentration, (5) Diffusion coefficient')
    user_input = input("Enter a list of integers (separated by commas): ")
    input_features = user_input.split(',')

    req_additional_features = []
    for feature in input_features:
        if feature not in ['0','1','2','3','4','5']:
            print('Invalid input. Please enter a number')
            req_additional_features.clear()
            break
        else:
            req_additional_features.append(additional_features[int(feature)])

    df_processed = df_processed.reindex(columns=[*req_additional_features, *df_processed.columns])
    df_processed.fillna(0, inplace=True)


    for i in range(0, len(df)):
        row_index = i*replicates
        df_processed.loc[row_index:row_index+12, req_additional_features] = df[req_additional_features].iloc[i].values

    return df_processed 


In [13]:
def dataframe_to_excel(df_processed, file_path):
    
    print(user_input_additional_features, user_input_processing_type)
    # Get the current directory path
    curr_dir = os.getcwd()
    file_str = 'Dataset-LOXIMVIs-'
    
    if user_input_additional_features == 1:
        file_str += 'Additional-Features-'
    else:
        file_str = file_str

    print(file_str)
    file_path = os.path.join(curr_dir, file_str)
    df_processed.to_excel(file_path, index=False, engine='openpyxl')


In [14]:
print('Enter the type of data processing to be done: ')
print('1. Formulation Composition')
print('2. One Hot Encoding: Lipid Composition')
print('3. One Hot Encoding: Head and Tail Composition')
try:
    user_input_processing_type = input('Enter the number corresponding to the data processing type and hit enter: ')
    if user_input_processing_type == '1':
        df_processed = form_composition_matrix(df, replicates, target)
        file_str += 'Formulation-Composition'
    elif user_input_processing_type == '2':
        df_processed = one_hot_lipid_comp(df, replicates, target)
        file_str += 'Lipid-Composition'
    elif user_input_processing_type == '3':
        df_processed = one_hot_head_tail_comp(df, replicates, target)
        file_str += 'Head-Tail-Composition'
    else:
        print('Invalid input. Please enter a valid number')
        exit()

    # Remove rows with 0 in the last column
    df_processed = df_processed[df_processed.iloc[:, -1] != 0]
    
    print('Do you want to add more features?')
    print('1. Yes')
    print('2. No')
    user_input_additional_features = input('Enter the number corresponding to your choice and hit enter: ')

    if user_input_additional_features == '1':
        df_processed = add_more_features(df, df_processed, replicates)
        file_str += '-Additional-Features.xlsx'
        
    elif user_input_additional_features == '2':
        print('No additional features added')
        file_str += '.xlsx'

    # Creating file path and making output
    curr_dir = os.getcwd()
    file_path_new = os.path.join(curr_dir, file_str)

    print(file_str)
    df_processed.to_excel(file_path_new, index=False, engine='openpyxl')
except:
    print('Unexpected error. Please enter a valid number')

Enter the type of data processing to be done: 
1. Formulation Composition
2. One Hot Encoding: Lipid Composition
3. One Hot Encoding: Head and Tail Composition
Do you want to add more features?
1. Yes
2. No
Enter a list of integers separated by commas: for denoting the columns to be used. Ex: 0,1,2,3,4,5
(0)ID (1) Z-Avg, (2) Zeta Potential, (3) PDI, (4) Concentration, (5) Diffusion coefficient
Dataset-LOXIMVIs-Head-Tail-Composition-Additional-Features.xlsx
